# <center>Implementing LDA in Python</center>

## Introduction

## Importing the Required Libraries

In [131]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))

In [98]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import pandas as pd
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [99]:
keywords = pd.read_csv("ner_nlp.csv")

In [100]:
lista =keywords['Author Keywords'].dropna()

In [101]:
lista3 = [item.replace("; ", ",").replace("/", " ") for item in lista]

In [102]:
lista3 =[item.replace("-", " ") for item in lista3]


In [103]:
lista3


['Entity disambiguation,Information extraction,Microblogs,Named entity recognition,Twitter',
 'Clinical text processing,Machine learning,Natural language processing',
 'Biomedical named entity recognition,Deep neural networks,Natural language processing',
 'Chinese Clinical Text,Clinical Natural Language Processing,Deep Learning,Named Entity Recognition,Neural Network',
 'Clinical notes,Deep learning,Entity recognition,Recurrent neural network,Sequence labeling',
 'Discriminative sequence classifier,Information extraction,Natural language processing,Radiology report narrative',
 'Biomedical text mining,Convolutional neural networks,Multi task learning,Named entity recognition',
 'Clinical notes,Disorder identifciation,Information extraction,Named entity recognition,Natural language processing,Word sense disambiguation',
 'Information extraction,Named entity recognition,Relation extraction,Text mining',
 'Electronic health records,Information extraction,Natural language processing',
 'A

In [104]:
key = []
for item in lista3:
    key.append(item.lower().split(','))
    
key

[['entity disambiguation',
  'information extraction',
  'microblogs',
  'named entity recognition',
  'twitter'],
 ['clinical text processing',
  'machine learning',
  'natural language processing'],
 ['biomedical named entity recognition',
  'deep neural networks',
  'natural language processing'],
 ['chinese clinical text',
  'clinical natural language processing',
  'deep learning',
  'named entity recognition',
  'neural network'],
 ['clinical notes',
  'deep learning',
  'entity recognition',
  'recurrent neural network',
  'sequence labeling'],
 ['discriminative sequence classifier',
  'information extraction',
  'natural language processing',
  'radiology report narrative'],
 ['biomedical text mining',
  'convolutional neural networks',
  'multi task learning',
  'named entity recognition'],
 ['clinical notes',
  'disorder identifciation',
  'information extraction',
  'named entity recognition',
  'natural language processing',
  'word sense disambiguation'],
 ['information ex

In [105]:
for i in key:
    for j in i:
        if len(j) ==3: 
            for k in i:
                if j[0] == k.split(' ')[0]:
                    print (j, k)
                

ner n grams


In [106]:
#eliminando parenteses e conteudo dos parenteses
clean1 = []
for i, v in enumerate(key):
    for j, k in enumerate(v):
         if '(' in k:
            key[i][j] = re.sub(r' *\([a-z0-9]* *[a-z0-9].*\)','', key[i][j])
  


In [107]:
key[12][2]

'named entity recognition'

In [108]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(key)

texts = key

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

In [109]:


low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1)],
 [(7, 1), (8, 1), (9, 1)],
 [(3, 1), (10, 1), (11, 1), (12, 1), (13, 1)],
 [(12, 1), (14, 1), (15, 1), (16, 1), (17, 1)],
 [(1, 1), (7, 1), (18, 1), (19, 1)],
 [(3, 1), (20, 1), (21, 1), (22, 1)],
 [(1, 1), (3, 1), (7, 1), (14, 1), (23, 1), (24, 1)],
 [(1, 1), (3, 1), (25, 1), (26, 1)],
 [(1, 1), (7, 1), (27, 1)],
 [(7, 1), (28, 1), (29, 1), (30, 1), (31, 1)],
 [(1, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1)],
 [(3, 1), (7, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)],
 [(3, 1), (7, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)],
 [(3, 1), (52, 1), (53, 1), (54, 1), (55, 1)],
 [(6, 1), (11, 1), (56, 1), (57, 1)],
 [(27, 1), (57, 1), (58, 1), (59, 1)],
 [(60, 1), (61, 1), (62, 1), (63, 1), (64, 1)],
 [(8, 1), (16, 1), (65, 1), (66, 1), (67, 1)],
 [(1, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1)],


In [110]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [124]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [125]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.20615944), (1, 0.30784142), (2, 0.033093985), (3, 0.27403557), (4, 0.03633621), (5, 0.035180032), (6, 0.024634805), (7, 0.042195015), (8, 0.01440264), (9, 0.026120847)]
[(1, 0.30784142), (3, 0.27403557), (0, 0.20615944), (7, 0.042195015), (4, 0.03633621), (5, 0.035180032), (2, 0.033093985), (9, 0.026120847), (6, 0.024634805), (8, 0.01440264)]


In [126]:
lda_model.save("data/test_model.model")

In [127]:
new_model = gensim.models.ldamodel.LdaModel.load("data/test_model.model")

In [128]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.20616038), (1, 0.3078432), (2, 0.033094097), (3, 0.27403218), (4, 0.03633634), (5, 0.035180155), (6, 0.02463489), (7, 0.042195175), (8, 0.014402689), (9, 0.026120938)]
[(1, 0.3078432), (3, 0.27403218), (0, 0.20616038), (7, 0.042195175), (4, 0.03633634), (5, 0.035180155), (2, 0.033094097), (9, 0.026120938), (6, 0.02463489), (8, 0.014402689)]


## Vizualizing the Data

In [129]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)


D:\Users\leobm\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return pd.DataFrame({'Term': vocab[term_ix], \


In [130]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.229860  0.336389       1        1  38.658856
1      0.168839 -0.310532       2        1  12.757966
7      0.080875  0.234444       3        1   9.015480
4     -0.077685 -0.222665       4        1   7.557873
5     -0.155634  0.068422       5        1   7.358760
2      0.283066  0.141901       6        1   6.251657
6      0.196270 -0.062264       7        1   5.993273
9     -0.319843 -0.127539       8        1   5.017550
3      0.061111 -0.002659       9        1   3.907832
8     -0.007138 -0.055497      10        1   3.480753, topic_info=                             Term         Freq        Total Category  logprob  \
3        named entity recognition  1097.000000  1097.000000  Default  10.0000   
7     natural language processing   537.000000   537.000000  Default   9.0000   
1          information extraction   165.000000   165.000000  Default   8.0000   
6                machine learning   148.000000   148.000000  Default   7.0000   
751                      ontology    71.000000    71.000000  Default   6.0000   
...                           ...          ...          ...      ...      ...   
427                 bootstrapping     3.807747     4.437609  Topic10  -4.2554   
910             domain adaptation     3.178258     3.808097  Topic10  -4.4361   
2275           feature extraction     2.518702     3.148581  Topic10  -4.6687   
2657                   noisy data     2.381139     3.011243  Topic10  -4.7249   
624        biomedical informatics     2.288021     2.917922  Topic10  -4.7648   

      loglift  
3     10.0000  
7      9.0000  
1      8.0000  
6      7.0000  
751    6.0000  
...       ...  
427    3.2048  
910    3.1771  
2275   3.1347  
2657   3.1231  
624    3.1147  

[111 rows x 6 columns], token_table=      Topic      Freq                       Term
term                                            
2978      8  0.970384     analysis of news flows
271       5  0.954022    artificial intelligence
375       3  0.967453                       bert
183       3  0.984620                    bi lstm
272       5  0.828477                bi lstm crf
...     ...       ...                        ...
189       8  0.903331             word embedding
67       10  0.974146            word embeddings
638       3  0.858369        word representation
24        2  0.924602  word sense disambiguation
182       6  0.975698                   word2vec

[102 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 8, 5, 6, 3, 7, 10, 4, 9])